Imports

In [1]:
# Configuration file for the ML Food Buddy Recommender project
import os
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
import re

Functions

In [2]:
# Create data loader function
def data_loader(filename: str, source: str = "raw"):
    """
    Load a CSV dataset from the repo's data/raw or data/preprocessed folder.
    
    Parameters:
        filename: str - the CSV file name (can be compressed .zip or .gz)
        source: str - "raw" or "preprocessed" to select folder
    
    Returns:
        pd.DataFrame
    """
    if source not in ["raw", "preprocessed"]:
        raise ValueError("source must be 'raw' or 'preprocessed'")

    cwd = os.getcwd()
    repo_root = cwd

    # Walk upwards until we find the desired folder
    while True:
        data_path = os.path.join(repo_root, "data", source, filename)
        if os.path.exists(data_path):
            break
        parent = os.path.dirname(repo_root)
        if parent == repo_root:  # reached root of filesystem
            raise FileNotFoundError(f"Could not find {filename} in data/{source} from {cwd}")
        repo_root = parent

    # Detect compression type
    compression_type = None
    if filename.endswith('.zip'):
        compression_type = 'zip'
    elif filename.endswith('.gz'):
        compression_type = 'gzip'

    return pd.read_csv(data_path, compression=compression_type)


In [3]:
# Create parse time function
def parse_time(t):
    """
    Parse time in minutes. Supports:
    - Raw numeric strings (e.g., "45")
    - Already numeric values
    
    Returns:
        float (minutes) or np.nan if parsing fails
    """
    if pd.isna(t):
        return np.nan
    if isinstance(t, str):
        t = t.strip()
        if t.startswith("PT"):  # ISO 8601 duration
            hours = re.search(r'(\d+)H', t)
            minutes = re.search(r'(\d+)M', t)
            secs = re.search(r'(\d+)S', t)
            total_minutes = 0
            if hours:
                total_minutes += int(hours.group(1)) * 60
            if minutes:
                total_minutes += int(minutes.group(1))
            if secs:
                total_minutes += int(secs.group(1)) / 60
            return total_minutes if total_minutes > 0 else np.nan
        # fallback: try to parse as float
        try:
            return float(t)
        except:
            return np.nan
    # If already numeric
    try:
        return float(t)
    except:
        return np.nan


In [4]:
def clip_top_outliers(df, cols, z_thresh=3.0):
    """
    Clip the top extreme values of selected columns using robust Z-score.
    
    Parameters:
    - df: pd.DataFrame
    - cols: list of str, columns to clip
    - z_thresh: float, threshold for robust Z-score (default=3)
    
    Returns:
    - df_clipped: pd.DataFrame with clipped values
    - thresholds: dict of column:clip_value for reference
    """
    df_clipped = df.copy()
    thresholds = {}
    
    for col in cols:
        if col not in df.columns:
            continue
        series = df[col]
        median = series.median()
        mad = np.median(np.abs(series - median))
        if mad == 0:
            # fallback to normal std if MAD is zero
            mad = series.std() * 1.4826  # approximate MAD
        z_score = (series - median) / (mad + 1e-10)
        upper_limit = median + z_thresh * mad
        df_clipped[col] = np.minimum(series, upper_limit)
        thresholds[col] = upper_limit
    
    return df_clipped, thresholds

EDA

In [5]:
# Load the dataset
recipes = data_loader("recipes.csv")
recipes.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc..."
4,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"c(""Mix everything together and bring to a boil..."


In [6]:
# info
recipes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522517 entries, 0 to 522516
Data columns (total 28 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   RecipeId                    522517 non-null  int64  
 1   Name                        522517 non-null  object 
 2   AuthorId                    522517 non-null  int64  
 3   AuthorName                  522517 non-null  object 
 4   CookTime                    439972 non-null  object 
 5   PrepTime                    522517 non-null  object 
 6   TotalTime                   522517 non-null  object 
 7   DatePublished               522517 non-null  object 
 8   Description                 522512 non-null  object 
 9   Images                      522516 non-null  object 
 10  RecipeCategory              521766 non-null  object 
 11  Keywords                    505280 non-null  object 
 12  RecipeIngredientQuantities  522514 non-null  object 
 13  RecipeIngredie

In [7]:
# Convert PrepTime into minutes
recipes['PrepTime_min'] = recipes['PrepTime'].apply(parse_time)

# Convert CookTime into minutes
recipes['CookTime_min'] = recipes['CookTime'].apply(parse_time)

# Convert TotalTime into minutes
recipes['TotalTime_min'] = recipes['TotalTime'].apply(parse_time)

In [8]:
# Identify unparseable PrepTime entries
invalid_prep_times = recipes[recipes['PrepTime'].apply(parse_time).isna()][['PrepTime']]
print("Invalid or unparseable PrepTime entries:")
print(invalid_prep_times.value_counts())

# Identify unparseable TotalTime entries
invalid_cooking_times = recipes[recipes['CookTime'].apply(parse_time).isna()][['CookTime']]
print()
print("Invalid or unparseable CoolTime entries:")
print(invalid_cooking_times.value_counts())

# Identify unparseable TotalTime entries
invalid_total_times = recipes[recipes['TotalTime'].apply(parse_time).isna()][['TotalTime']]
print()
print("Invalid or unparseable TotalTime entries:")
print(invalid_total_times.value_counts())

Invalid or unparseable PrepTime entries:
PrepTime
PT0S        15010
Name: count, dtype: int64

Invalid or unparseable CoolTime entries:
Series([], Name: count, dtype: int64)

Invalid or unparseable TotalTime entries:
TotalTime
PT0S         2129
Name: count, dtype: int64


In [9]:
# Convert DatePublished to datetime
recipes['DatePublished'] = pd.to_datetime(recipes['DatePublished'], errors='coerce')

In [ ]:
# Remove timezone to make tz-naive
recipes['DatePublished'] = recipes['DatePublished'].dt.tz_convert(None)

# Identify unparseable dates
invalid_dates = recipes[recipes['DatePublished'].isna()][['DatePublished']]
print("Invalid or unparseable dates:")
print(invalid_dates.value_counts())

# Define a reasonable date range
today = pd.Timestamp.today()
min_reasonable_date = pd.Timestamp('1900-01-01')

# Identify dates outside reasonable range
unreasonable_dates = recipes[(recipes['DatePublished'] < min_reasonable_date) |
                             (recipes['DatePublished'] > today)][['DatePublished']]
print("Unreasonable dates:")
print(unreasonable_dates.value_counts())


Invalid or unparseable dates:
Series([], Name: count, dtype: int64)
Unreasonable dates:
Series([], Name: count, dtype: int64)


**Note:**  
The entries for Prep and Cook time identified as invalid or unparseable are likely incorrect or missing.  
It is recommended to replace these entries with the **median** or **average** value of the respective column to avoid skewing any analysis.

In [11]:
# Missing values percentage, excluding columns with 0% missing
missing_pct = recipes.isna().sum() / len(recipes) * 100
missing_pct = missing_pct[missing_pct > 0].sort_values(ascending=False)
print(missing_pct)

RecipeYield                   66.614292
AggregatedRating              48.462155
ReviewCount                   47.364775
RecipeServings                35.005751
CookTime                      15.797572
CookTime_min                  15.797572
Keywords                       3.298840
PrepTime_min                   2.872634
TotalTime_min                  0.407451
RecipeCategory                 0.143727
Description                    0.000957
RecipeIngredientQuantities     0.000574
Images                         0.000191
dtype: float64


In [12]:
# Display summary statistics for all
print(recipes.describe())

            RecipeId      AuthorId                  DatePublished  \
count  522517.000000  5.225170e+05                         522517   
mean   271821.436970  4.572585e+07  2008-01-18 06:34:47.071884800   
min        38.000000  2.700000e+01            1999-08-06 00:40:00   
25%    137206.000000  6.947400e+04            2005-09-13 10:25:00   
50%    271758.000000  2.389370e+05            2007-12-13 16:27:00   
75%    406145.000000  5.658280e+05            2009-12-31 09:45:00   
max    541383.000000  2.002886e+09            2020-12-22 22:12:00   
std    155495.878422  2.929714e+08                            NaN   

       AggregatedRating    ReviewCount       Calories     FatContent  \
count     269294.000000  275028.000000  522517.000000  522517.000000   
mean           4.632014       5.227784     484.438580      24.614922   
min            1.000000       1.000000       0.000000       0.000000   
25%            4.500000       1.000000     174.200000       5.600000   
50%            5.0

**Dataset Summary Observations:**

- **Extreme scales & outliers:**`Calories`, `FatContent`, `SodiumContent` max far above 75th percentile - likely data errors.  
- **Skewed distributions:** Means > medians for nutrition columns - right-skewed, few extreme values pull averages up.  
- **Missing data:** `AggregatedRating`, `ReviewCount`, `RecipeServings`, `RecipeYield` and 'CookTime' have many NaNs.
- **Time inconsistencies:** `PrepTime_min` max = 1,440,000 - unrealistic.  

In [13]:
# Columns to clip
numeric_cols_to_clip = [
    'AggregatedRating', 'ReviewCount', 'Calories', 'FatContent',
    'SaturatedFatContent', 'CholesterolContent', 'SodiumContent',
    'CarbohydrateContent', 'FiberContent', 'SugarContent',
    'ProteinContent', 'RecipeServings', 'PrepTime_min'
]

# Apply clipping
recipes_clipped, clip_thresholds = clip_top_outliers(recipes, numeric_cols_to_clip)

# Show thresholds
for col, val in clip_thresholds.items():
    print(f"{col} clipped at: {val:.2f}")


AggregatedRating clipped at: nan
ReviewCount clipped at: nan
Calories clipped at: 812.10
FatContent clipped at: 42.90
SaturatedFatContent clipped at: 16.10
CholesterolContent clipped at: 170.40
SodiumContent clipped at: 1185.20
CarbohydrateContent clipped at: 81.30
FiberContent clipped at: 7.00
SugarContent clipped at: 21.70
ProteinContent clipped at: 31.00
RecipeServings clipped at: nan
PrepTime_min clipped at: nan
